In [15]:
import pandas as pd
import os 
os.chdir(r'./')
import itertools
df_Votes = pd.read_csv('BAF-al_2020.csv',dtype={'GEOID20_left':str,'COUNTYFP20':str})

 ''' Uncomment read_csv file to generate results 
     comapring current Alabama VRA district to the 2 
     proposed in Whole County Map '''

df_CD = pd.read_csv('AL_94_CD116_7.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})
#df_CD = pd.read_csv('AL_94_WC_6.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})
#df_CD = pd.read_csv('AL_94_WC_7.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})

In [16]:


df_Votes = df_Votes[['VTDST20','COUNTYFP20','G20PRERTRU','G20PREDBID']]
df_Votes = df_Votes.drop_duplicates().reset_index(drop=True)
df_Votes = df_Votes.rename(columns={'VTDST20':'VTD','COUNTYFP20':'COUNTY'})

df_CD = df_CD[['VTD','COUNTY','P0010001','P0010004']]
df_CD = df_CD.groupby(['VTD','COUNTY']).sum()
df = df_CD.merge(df_Votes, how='inner',on=['VTD','COUNTY'])
df = df.drop_duplicates()
df = df.drop(columns=['COUNTY'])
df_sum = df.groupby(['VTD']).sum().reset_index()
df = df[['VTD']].merge(df_sum, on='VTD')
df = df.drop_duplicates()
assert len(df.index) != 0
assert len(df_Votes.index) != 0

,VTD,P0010001,P0010004,G20PRERTRU,G20PREDBID
0,000001,2385,781,627,417
4,000002,2063,576,710,333
8,000003,12122,7972,2062,3489
15,000004,2886,896,967,371
20,000005,3522,421,1512,399


In [23]:
df['Black_Percentage'] = df['P0010004']/df['P0010001']
df['Group'] = None
df = df.reset_index(drop=True)

for i ,row in df.iterrows():
    if row['Black_Percentage'] >= .9:
        df['Group'].iloc[i] = 'B90%'
    if row['Black_Percentage'] < .9:
        df['Group'].iloc[i] = 'B60%'
    if row['Black_Percentage'] < .6:
        df['Group'].iloc[i] = 'N50%'
    if row['Black_Percentage'] <= .4:
        df['Group'].iloc[i] = 'W60%'
    if row['Black_Percentage'] <= .1:
        df['Group'].iloc[i] = 'W90%'

quoat = 0
quoat = df[['G20PRERTRU','G20PREDBID']].sum()
quoat = (quoat['G20PREDBID'] + quoat['G20PRERTRU'])/2
voteGroups = df[['Group','G20PREDBID']].groupby('Group').sum().reset_index()
voteGroups = voteGroups.values.tolist()

In [25]:
idx = 0
output = {}
for i in range(1,len(voteGroups)+1):
    for j in itertools.combinations(voteGroups,i):
        c = 0
        for k in j:
            c += k[1]
            if c > quoat:
                idx += 1
                output[idx] = {}
                output[idx]['Coliltion'] = j
                output[idx]['Total'] = c
                break
assert bool(output) == True
crit_count = {'B90%':0,'B60%':0,'W90%':0,'W60%':0,'N50%':0}
for key, value in output.items():
    wC = value['Total']
    for i in value['Coliltion']:
        if wC - i[1] < quoat:
            crit_count[i[0]] += 1

df_share = pd.DataFrame(list(crit_count.items()),columns = ['Cat','B_Index'])
B_IdxSum = df_share['B_Index'].sum()
df_share['B_PowerShare'] = (df_share['B_Index']/B_IdxSum) * 100
df_share['B_Critical_Vote'] = (df_share['B_Index']/(2**(5-1)))*100
df_share

,Cat,B_Index,B_PowerShare,B_Critical_Vote
0,B90%,2,25.0,12.5
1,B60%,2,25.0,12.5
2,W90%,0,0.0,0.0
3,W60%,2,25.0,12.5
4,N50%,2,25.0,12.5
